In [50]:
import numpy as np
from math import log

# pretty print for np.array 
# from https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters/53164538#53164538
def pretty_print(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c' + u'\u2500' * w + u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i, AAA in enumerate(AA[:-1]):
                w1 = max([len(str(s)) for s in A[:, i]])
                print(str(AAA) + ' ' * (w1  - len(str(AAA)) + 1), end='')
            w1 = max([len(str(s)) for s in A[:, -1]])
            print(str(AA[-1]) + ' ' * (w1 - len(str(AA[-1]))), end='')
            print(']')
        print(u'\u2514'+u'\u2500' * w + u'\u2518')  

def compare(left: set, right: set):
    if len(left) == len(right):
        for i in range(len(left)):
            if left[i] != right[i]:
                return left[i] < right[i]
    return left < right


### Generate $\mathcal{A}_n^2$

> Let $|\cdot|$ denotes cardinality and $\Delta$ denote symmetric different

In [51]:
def Delta(left: set, right: set):
    return left.symmetric_difference(right)

def Cardi(x: set):
    return len(x)

> Let $n=2^m$, $\Omega$ be a set of $m$ element such that $|\alpha_i| \leq |\alpha_{i+1}|$ and $|\alpha_i \Delta \alpha_{i+1}| \leq 2$  
> Let $\alpha_0=\{\varnothing\}$, now genereate $\Omega$

In [52]:
from itertools import chain, combinations

# param
m = 4
# generate \Omega

def legal(alpha_i: set, alpha_i_1: set):
    return Cardi(alpha_i) <= Cardi(alpha_i_1) and Cardi(Delta(alpha_i, alpha_i_1)) <= 2

def assert_omega(ome, _m, skip=0):
    for i in range(0 + skip, len(ome) - 1):
        assert(legal(ome[i], ome[i + 1]))
    assert(len(ome) == 2**_m + 1)

def insert_one_element(initial_sets: list, super_end: int) -> list:
    res = []
    for initial_set in initial_sets:
        avail_range = list(range(max(initial_set) + 1, super_end + 1))
        if len(avail_range) == 0:
            continue
        if len(res) == 0 and avail_range[0] not in initial_sets[-1]:
            avail_range.reverse()
        elif len(res) != 0 and avail_range[0] not in res[-1]:
            avail_range.reverse()
        for avail_ele in avail_range:
            if avail_ele in initial_set:
                break
            cur_res = initial_set.copy()
            cur_res.add(avail_ele)
            res.append(cur_res)
    
    return res

def create_Omega(_m: int):
    initial_sets = [{i + 1} for i in range(_m)]
    grouped_res = [initial_sets]
    for _ in range(1, _m):
        grouped_res.append(insert_one_element(grouped_res[-1], _m))

    res = [set(), set()]
    for single_res in  grouped_res:
        res.extend(single_res)
    s = list(range(1, _m + 1))
    unoredered = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
    for ele in unoredered:
        assert(set(ele) in res)
    assert(len(unoredered) == len(res) - 1)
    assert_omega(res, _m)
    return res

omega = create_Omega(m)
omega

[set(),
 set(),
 {1},
 {2},
 {3},
 {4},
 {1, 4},
 {1, 3},
 {1, 2},
 {2, 4},
 {2, 3},
 {3, 4},
 {1, 3, 4},
 {1, 2, 3},
 {1, 2, 4},
 {2, 3, 4},
 {1, 2, 3, 4}]

> $$a_{ij} = \begin{cases}
    -1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\alpha_{i-1}\Delta\alpha_{i}\;and\;|\alpha_{i-1}\Delta\alpha_{i}|=2 \\
    (-1)^{|\alpha_{i-1}\bigcap\alpha_j| + 1},\;\alpha_{j}\bigcap(\alpha_{i-1}\bigcup\alpha_{i})\neq\varnothing\;but\;does\;not\;meet\;the\;condition\;above \\
    1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\varnothing \\
\end{cases}
$$

In [53]:
def query_element(i: int, j: int, _omega: list) -> int:
    alpha_j = _omega[j + 1]
    alpha_i_1 = _omega[i]
    alpha_i = _omega[i + 1]

    if alpha_j.intersection(alpha_i_1.union(alpha_i)) == Delta(alpha_i_1, alpha_i) \
        and Cardi(Delta(alpha_i_1, alpha_i)) == 2:
        return -1
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) != 0:
        return (-1)**(Cardi(alpha_i_1.intersection(alpha_j)) + 1)
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) == 0:
        return 1
    else:
        raise ValueError("Undefined behavior!")
    
def create_A(_omega, _m):
    A_mat = np.zeros((2**_m, 2**_m))
    for i in range(A_mat.shape[0]):
        for j in range(A_mat.shape[1]):
            A_mat[i, j] = query_element(i, j, _omega)
    return A_mat

A_mat = create_A(omega, m)
pretty_print(A_mat.astype(int))

┌─────────────────────────────────────────────────┐
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 1  1  1  -1 -1 -1 1  1  1  -1 -1 -1 1  -1]
 [1 1  -1 1  1  1  1  -1 -1 -1 1  1  -1 -1 -1 -1]
 [1 1  1  -1 1  1  -1 1  1  -1 -1 -1 -1 1  -1 -1]
 [1 1  1  1  -1 -1 1  1  -1 1  -1 -1 1  -1 -1 -1]
 [1 -1 1  1  1  1  -1 -1 1  1  1  1  -1 1  1  1 ]
 [1 1  1  -1 1  -1 1  1  1  -1 -1 -1 1  -1 -1 -1]
 [1 1  -1 1  1  1  -1 1  -1 -1 1  -1 -1 1  -1 -1]
 [1 1  1  1  -1 -1 1  -1 1  1  -1 -1 -1 -1 1  -1]
 [1 1  1  -1 1  1  -1 1  -1 1  -1 -1 1  -1 -1 -1]
 [1 1  1  1  -1 -1 1  1  -1 -1 1  1  -1 -1 -1 -1]
 [1 -1 1  1  1  1  1  -1 1  1  -1 -1 1  1  -1 -1]
 [1 1  -1 1  1  -1 -1 1  -1 1  -1 1  -1 -1 -1 1 ]
 [1 1  1  1  -1 1  -1 -1 1  -1 -1 -1 1  -1 -1 1 ]
 [1 1  1  -1 1  -1 -1 -1 -1 1  1  -1 -1 1  -1 1 ]
 [1 -1 1  1  1  1  1  1  -1 -1 -1 -1 -1 -1 1  1 ]
└─────────────────────────────────────────────────┘


### Check $A = LQ$

> Let $Q$ be a $n$ by $n$ matrix given by $q_{ij} = (-1)^{|\alpha_i \bigcap \alpha_j|}$, $Q$ is a symmetric Hadamard matrix, that is $Q^2=QQ^T=nI_n$

In [54]:
def get_Q(_omega, _n):
    Q_mat = np.zeros((_n, _n))
    for i in range(_n):
        for j in range(_n):
            Q_mat[i, j] = (-1) ** Cardi(_omega[i + 1].intersection(_omega[j + 1]))
    return np.matrix(Q_mat)

Q_mat = get_Q(omega, 2**m)
pretty_print(np.array((Q_mat * Q_mat).astype(int)))

┌─────────────────────────────────────────────────┐
 [16 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  16 0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  16 0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  16 0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  16 0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  16 0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  16 0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  16 0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  16 0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  16 0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  16 0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  16 0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  16 0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  16 0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  0  16 0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  16]
└─────────────────────────────────────────────────┘


> Simply verify matrix $L$ is a lower triangular matrix

In [56]:
pretty_print(np.array(np.matrix(A_mat) * np.matrix(Q_mat).I))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
 [1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.5   0.5   0.0   0.0   -0.5  0.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   0.5   0.5   -0.5  0.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.5   0.5   0.0   0.0   -0.5  0.5   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.5   0.0   0.0   0.5   0.0   0.0   -0.5  0.5   0.0   0.0   0.0   0.0   0.0

### Generate $\mathcal{A}_{n - 1}^1$

> Let $B\in\mathcal{A}_{n - 1}^1$, $\Phi(B)\in\mathcal{A}_{n}^2\;$:
> $$ \Phi(B) = \left(\begin{array}{cc} 
1 & 1_{n-1}\\
-1^T_{n-1} & 2B-J_{n-1}
\end{array}\right)
> $$
> And: 
> $$ \Phi(B)^{-1} = \left(\begin{array}{cc} 
1-\frac{1}{2}1_{n-1}B^{-1}1^T_{n-1} & -\frac{1}{2}1_{n-1}B^{-1}\\
\frac{1}{2}B^{-1}1^T_{n-1} & \frac{1}{2}B^{-1}
\end{array}\right)
> $$

In [57]:
def create_A_1_mat(A_2_mat):
    A_1_mat = A_2_mat.copy()
    for i in range(1, A_1_mat.shape[0]):
        A_1_mat[i, :] = A_1_mat[i, :] * -1

    for j in range(1, A_1_mat.shape[1]):
        if A_1_mat[0, j] == -1:
            A_1_mat[:, j] = A_1_mat[:, j] * -1

    A_1_mat = (A_1_mat[1:, 1:] + np.ones(2**m - 1)) * 0.5
    assert(A_1_mat.max() == 1 and A_1_mat.min() == 0)
    return A_1_mat

pretty_print(create_A_1_mat(A_mat).astype(int))

┌───────────────────────────────┐
 [1 0 0 0 1 1 1 0 0 0 1 1 1 0 1]
 [0 1 0 0 0 0 1 1 1 0 0 1 1 1 1]
 [0 0 1 0 0 1 0 0 1 1 1 1 0 1 1]
 [0 0 0 1 1 0 0 1 0 1 1 0 1 1 1]
 [1 0 0 0 0 1 1 0 0 0 0 1 0 0 0]
 [0 0 1 0 1 0 0 0 1 1 1 0 1 1 1]
 [0 1 0 0 0 1 0 1 1 0 1 1 0 1 1]
 [0 0 0 1 1 0 1 0 0 1 1 1 1 0 1]
 [0 0 1 0 0 1 0 1 0 1 1 0 1 1 1]
 [0 0 0 1 1 0 0 1 1 0 0 1 1 1 1]
 [1 0 0 0 0 0 1 0 0 1 1 0 0 1 1]
 [0 1 0 0 1 1 0 1 0 1 0 1 1 1 0]
 [0 0 0 1 0 1 1 0 1 1 1 0 1 1 0]
 [0 0 1 0 1 1 1 1 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 1 1 1 1 1 1 0 0]
└───────────────────────────────┘


In [58]:
def funcX(A):
    return np.absolute(np.matrix(A).I).max()

def get_o_1(_X, _n):
    return (0.5 * _n * log(_n, 2) - _n - log(_X, 2)) / _n

### Verify $χ(A)=2^{\frac{1}{2}nlogn - n(1+o(1))}$

In [62]:
for m in range(2, 7):
    omega = create_Omega(m)
    A_2_mat = create_A(omega, m)
    X_A_2 = funcX(A_2_mat)
    n = A_2_mat.shape[0]
    o_1 = get_o_1(X_A_2, n)
    print("m = {m_term}, n = {n_term}, χ(A_2) = {X_A_2_term}, o(1) term = {o_1_term:.3f}".format(m_term=m, n_term=n, X_A_2_term=X_A_2, o_1_term=o_1))

m = 2, n = 4, χ(A_2) = 0.5, o(1) term = 0.250
m = 3, n = 8, χ(A_2) = 1.0, o(1) term = 0.500
m = 4, n = 16, χ(A_2) = 131.50000000000142, o(1) term = 0.560
m = 5, n = 32, χ(A_2) = 2247877518.8656816, o(1) term = 0.529
m = 6, n = 64, χ(A_2) = 3344658601622888.0, o(1) term = 1.194


### Verify $χ(A')=2^{\frac{1}{2}nlogn - n(1+o(1))}$

In [64]:
for m in range(2, 7):
    omega = create_Omega(m)
    assert_omega(omega, m)
    A_mat = create_A(omega, m)
    A_1_mat = create_A_1_mat(A_mat)
    X_A = funcX(A_1_mat)
    n = A_1_mat.shape[0] + 1
    o_1 = get_o_1(X_A, n)
    print("m = {m_term}, n = {n_term}, χ(A) = {X_A_term}, o(1) term = {o_1_term:.3f}".format(m_term=m, n_term=n, X_A_term=X_A, o_1_term=o_1))

m = 2, n = 4, χ(A) = 1.0, o(1) term = 0.000
m = 3, n = 8, χ(A) = 2.0, o(1) term = 0.375
m = 4, n = 16, χ(A) = 260.0000000000009, o(1) term = 0.499
m = 5, n = 32, χ(A) = 4495480119.287977, o(1) term = 0.498
m = 6, n = 64, χ(A) = 4212068385296025.0, o(1) term = 1.189


### Generate matrix $C$

In [65]:
def square_operator(S, T):
    top_right = np.zeros((S.shape[0], T.shape[1]),dtype=int)
    btn_left = np.zeros((T.shape[0], S.shape[1]),dtype=int)
    if btn_left.shape[1] > 0:
        btn_left[0, -1] = 1
    return np.asarray(np.bmat([[S, top_right], [btn_left, T]]))

res = None

for m in range(2, 5):
    omega = create_Omega(m)
    assert_omega(omega, m)
    A_mat = create_A(omega, m)
    A_1_mat = create_A_1_mat(A_mat)
    res = A_1_mat if res is None else square_operator(res, A_1_mat)

pretty_print(res.astype(int))

┌───────────────────────────────────────────────────┐
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1